In [65]:
%load_ext autoreload
%autoreload 2
import sys
import os
project_root = os.path.abspath("/Users/joschua/repos/harris_lab/hypnose/hypnose-analysis")
if project_root not in sys.path:
    sys.path.append(project_root)
import pandas as pd
import numpy as np 
import utils
from utils import *
from pathlib import Path
import harp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import aeon.io.video as video
from ipywidgets import widgets
from IPython.display import display
import re
import os
import zoneinfo
from datetime import datetime, timezone
from functools import reduce
from src.processing import detect_settings
import src.processing.detect_stage as detect_stage_module

%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
# Loading experiments --> just define the SUBJID and DATE
root = load_experiment(26, 20250902) #can add index for multiple experiments; index=0 as default
stage = detect_stage_module.detect_stage(root)

Using subject directory: /Volumes/harris/hypnose/rawdata/sub-026_id-077
Loaded experiment: /Volumes/harris/hypnose/rawdata/sub-026_id-077/ses-116_date-20250902/behav/2025-09-02T12-45-15
Detected stage: QuintupleProbeA_Location0
Hidden rule index: 0


In [67]:
data = load_all_streams(root)

Loading data streams from: /Volumes/harris/hypnose/rawdata/sub-026_id-077/ses-116_date-20250902/behav/2025-09-02T12-45-15
Loaded heartbeat data
Calculated real-time offset: 44411 days, 11:08:39
Created timestamp interpolation mapping
Loaded digital_input_data
Loaded output_set
Loaded output_clear
Loaded olfactometer_valves_0
Loaded olfactometer_valves_1
Loaded olfactometer_end_0
Loaded analog_data
Loaded flow_meter
Loaded video_data
Loaded pulse_supply_1
Loaded pulse_supply_2
Created odour_led

Applying time corrections to all data streams...
Applied correction to digital_input_data
Applied correction to output_set
Applied correction to output_clear
Applied correction to olfactometer_valves_0
Applied correction to olfactometer_valves_1
Applied correction to olfactometer_end_0
Applied correction to analog_data
Applied correction to flow_meter
Applied correction to video_data
Applied correction to pulse_supply_1
Applied correction to pulse_supply_2
Applied correction to odour_led

Data l

In [68]:
events = load_experiment_events(root)

Loading experiment events...
Loaded heartbeat data for timing synchronization
Calculated real-time offset: 44411 days, 11:08:39
Created timestamp interpolation mapping
Found 4 experiment event files
Processing event file: ExperimentEvents_1904-01-30T04-00-00.csv with 321 rows
Using Seconds column with interpolation
Applied real-time offset: 44411 days, 11:08:39
Found Value column with values: ['EndInitiation' 'Reset' 'ChooseRandomSequence' 'SampleRewardCondition'
 'InitiationSequence' 'AwaitReward']
Found 59 EndInitiation events
Found 58 InitiationSequence events
Found 59 Reset events
Found 29 AwaitReward events
Found 58 SampleRewardCondition events
Found 58 ChooseRandomSequence events
Processing event file: ExperimentEvents_1904-01-30T03-00-00.csv with 351 rows
Using Seconds column with interpolation
Applied real-time offset: 44411 days, 11:08:39
Found Value column with values: ['EndInitiation' 'AwaitReward' 'Reset' 'ChooseRandomSequence'
 'SampleRewardCondition' 'InitiationSequence']

In [69]:
odor_map = load_odor_mapping(root, data)
print("Data streams loaded:", list(data.keys()))
print("Event types loaded:", list(events.keys()))
print("Odor mapping:", odor_map['odour_to_olfactometer_map'])

Loading odor mapping from session settings...
Loaded session settings
Found 10 valve configurations for olfactometer 0
Found 10 valve configurations for olfactometer 1
Created valve-to-command mapping: {'00': 3, '01': 7, '02': 5, '03': 6, '10': 4, '11': 8, '12': 9, '13': 2}
Created odor mapping: [['OdorA', 'OdorE', 'OdorC', 'OdorD'], ['OdorB', 'OdorF', 'OdorG', 'Purge']]
Data streams loaded: ['heartbeat', 'real_time_offset', 'timestamp_to_time', 'interpolate_time', 'digital_input_data', 'output_set', 'output_clear', 'olfactometer_valves_0', 'olfactometer_valves_1', 'olfactometer_end_0', 'analog_data', 'flow_meter', 'video_reader', 'video_data', 'pulse_supply_1', 'pulse_supply_2', 'odour_led', 'behavior_reader', 'olfactometer_reader']
Event types loaded: ['combined_initiation_sequence_df', 'combined_end_initiation_df', 'combined_await_reward_df', 'combined_reset_df', 'combined_choose_random_sequence_df', 'combined_sample_reward_condition_df']
Odor mapping: [['OdorA', 'OdorE', 'OdorC', '

In [70]:
trial_counts = detect_trials(data, events, root, verbose=True)

TRIAL DETECTION - METHOD 3: Simplified
Parameters: minimum_sampling_time=350.0ms, sample_offset_time=200.0ms

InitiationSequence 0: 2025-09-02 13:47:23.232992
  Found 14 poke periods
    Starting attempt 1 at 2025-09-02 13:47:28.728096
      Poke 1: 410.0ms (total: 410.0ms)
      SUCCESS: 410.0ms continuous poke (≥350.0ms)

InitiationSequence 1: 2025-09-02 13:47:38.633984
  Found 12 poke periods
    Starting attempt 1 at 2025-09-02 13:47:44.878144
      Poke 1: 45.8ms (total: 45.8ms)
      Poke 2: gap 130.9ms + 134.7ms (total: 311.4ms)
      Poke 3: gap 170.2ms + 43.5ms (total: 525.1ms)
      SUCCESS: 525.1ms continuous poke (≥350.0ms)

InitiationSequence 2: 2025-09-02 13:47:55.070976
  Found 23 poke periods
    Starting attempt 1 at 2025-09-02 13:48:04.056832
      Poke 1: 348.9ms (total: 348.9ms)
      Gap to poke 2: 355.1ms (≥200.0ms - sequence ends)
      FAILED: 348.9ms continuous poke (<350.0ms)
    Starting attempt 2 at 2025-09-02 13:48:04.760896
      Poke 2: 121.4ms (total: 12

In [71]:
trial_outcomes_complete = classify_and_analyze_with_response_times(data, events, trial_counts, odor_map, stage, root)

CLASSIFYING TRIAL OUTCOMES (NO RESPONSE-TIME) WITH HIDDEN RULE AND VALVE/POKE TIME ANALYSIS
Sample offset time: 200.0 ms
Minimum sampling time: 350.0 ms
Response time window: 2.5 s
Hidden rule location extracted: Location0 (index 0, position 1)
Found 1150 total valve activations (excluding Purge)
Analyzing 248 initiated trials...
Found 159 AwaitReward events
Found 148 total supply port activities
Hidden Rule Odors inferred: ['OdorA', 'OdorB']

TRIAL CLASSIFICATION RESULTS WITH HIDDEN RULE AND VALVE/POKE TIME ANALYSIS (NO RESPONSE-TIME):
Hidden Rule Location: Position 1 (index 0)

Total attempts: 356
-- Non-initiated sequences: 108 (30.3%)
-- Initiated sequences (trials): 248 (69.7%)

INITIATED TRIALS BREAKDOWN:
-- Completed sequences: 158 (63.7%)
   -- Hidden Rule trials (HR): 13 (5.2%)
   -- Hidden Rule Missed (HR_missed): 6 (2.4%)
-- Aborted sequences: 90 (36.3%)
   -- Aborted Hidden Rule trials (HR): 7 (2.8%)

REWARD STATUS BREAKDOWN:
-- Rewarded: 147 (93.0%)
-- Unrewarded: 3 (1.9%)

# Testing of new functions

# Debugging Functions:



In [70]:
def check_data_availability(data, events, odor_map):
    """Quick check of all data streams for availability and content"""
    
    print("=" * 60)
    print("DATA AVAILABILITY CHECK")
    print("=" * 60)
    
    # Check main data streams
    print("\n📊 DATA STREAMS:")
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            has_data = not value.empty
            print(f"  {key}: {'✅ HAS DATA' if has_data else '❌ EMPTY'} ({value.shape})")
            if has_data:
                print(f"    Columns: {value.columns.tolist()}")
                print(f"    Sample:\n{value.head(3).to_string(max_cols=5)}")
                print()
        elif isinstance(value, pd.Series):
            has_data = not value.empty
            print(f"  {key}: {'✅ HAS DATA' if has_data else '❌ EMPTY'} ({len(value)} items)")
            if has_data:
                print(f"    Sample: {value.head(3).tolist()}")
                print()
        else:
            print(f"  {key}: {type(value).__name__}")
    
    # Check event streams
    print("\n🎯 EVENT STREAMS:")
    for key, value in events.items():
        if isinstance(value, pd.DataFrame):
            has_data = not value.empty
            print(f"  {key}: {'✅ HAS DATA' if has_data else '❌ EMPTY'} ({value.shape})")
            if has_data:
                print(f"    Columns: {value.columns.tolist()}")
                print(f"    Sample:\n{value.head(3).to_string()}")
                print()
    
    # Check odor mapping
    print("\n🌸 ODOR MAPPING:")
    print(f"  odour_to_olfactometer_map: {odor_map['odour_to_olfactometer_map']}")
    print(f"  valve_to_odor: {odor_map['valve_to_odor']}")
    
    # Check digital input ports specifically
    print("\n🔌 DIGITAL INPUT PORTS:")
    if 'digital_input_data' in data and not data['digital_input_data'].empty:
        digital_data = data['digital_input_data']
        for col in digital_data.columns:
            port_data = digital_data[col]
            has_activity = port_data.sum() > 0  # Check if any True values
            print(f"  {col}: {'✅ ACTIVE' if has_activity else '❌ NO ACTIVITY'} ({port_data.sum()} events)")
            if has_activity:
                # Show first few ON events
                on_events = port_data[port_data == True].head(3)
                print(f"    First ON events: {on_events.index.tolist()}")
    
    # Check output ports (LED control)
    print("\n💡 OUTPUT PORTS:")
    for stream_name in ['output_set', 'output_clear']:
        if stream_name in data and not data[stream_name].empty:
            output_data = data[stream_name]
            print(f"  {stream_name}:")
            for col in output_data.columns:
                port_data = output_data[col]
                has_activity = port_data.sum() > 0
                print(f"    {col}: {'✅ ACTIVE' if has_activity else '❌ NO ACTIVITY'} ({port_data.sum()} events)")

# Run the check
check_data_availability(data, events, odor_map)

DATA AVAILABILITY CHECK

📊 DATA STREAMS:
  heartbeat: ✅ HAS DATA ((9961, 2))
    Columns: ['Time', 'TimestampSeconds']
    Sample:
                 Time  TimestampSeconds
0 1904-01-12 03:42:00            963720
1 1904-01-12 03:42:01            963721
2 1904-01-12 03:42:02            963722

  real_time_offset: timedelta
  timestamp_to_time: ✅ HAS DATA (9961 items)
    Sample: [Timestamp('1904-01-12 03:42:00'), Timestamp('1904-01-12 03:42:01'), Timestamp('1904-01-12 03:42:02')]

  interpolate_time: function
  digital_input_data: ✅ HAS DATA ((6870, 4))
    Columns: ['DIPort0', 'DIPort1', 'DIPort2', 'DI3']
    Sample:
                            DIPort0  DIPort1  DIPort2    DI3
Time                                                        
2025-08-15 14:52:39.792832    False    False     True  False
2025-08-15 14:52:39.861536    False    False    False  False
2025-08-15 14:52:53.775296    False    False     True  False

  output_set: ✅ HAS DATA ((805, 14))
    Columns: ['DOPort0', 'DOPort1'

In [ ]:
# Detect short purge events. Probably needs a > 4 ms filter to avoid short purges in trial initiation
import pandas as pd

def list_short_purge_events(odor_map, threshold_ms=200.0):
    # uses _resolve_odor_name and _purge_columns defined above
    olfactometer_valves = odor_map["olfactometer_valves"]
    purge_cols = _purge_columns(odor_map)

    events = []
    for olf_id, idx, col in purge_cols:
        df = olfactometer_valves[olf_id]
        if df is None or getattr(df, "empty", True):
            continue
        s = df[col].astype(bool)

        rises = s & ~s.shift(1, fill_value=False)
        falls = ~s & s.shift(1, fill_value=False)
        starts = list(s.index[rises])
        ends = list(s.index[falls])

        i = j = 0
        while i < len(starts) and j < len(ends):
            # Skip stray falls before the first rise
            if ends[j] <= starts[i]:
                j += 1
                continue
            duration_ms = (ends[j] - starts[i]).total_seconds() * 1000.0
            if duration_ms < threshold_ms:
                events.append({
                    "olf_id": olf_id,
                    "valve_idx": idx,
                    "column": col,
                    "start": starts[i],
                    "end": ends[j],
                    "duration_ms": duration_ms,
                    "name": _resolve_odor_name(odor_map, olf_id, idx),
                })
            i += 1
            j += 1

    if not events:
        print(f"No Purge events shorter than {threshold_ms} ms found.")
        return

    events.sort(key=lambda e: e["start"])
    print(f"Purge events < {threshold_ms} ms (n={len(events)}):")
    for e in events:
        print(f"- olf {e['olf_id']} idx {e['valve_idx']} ({e['name']}), col '{e['column']}': "
              f"{e['start'].isoformat()} -> {e['end'].isoformat()}  "
              f"{e['duration_ms']:.1f} ms")

# Run it
list_short_purge_events(odor_map, threshold_ms=200.0)

In [ ]:
# Identify trial IDs for trials with poke time of < 350 ms 
threshold_ms = 350.0

# Get the classification DataFrames
cls = trial_outcomes_complete.get("classification", trial_outcomes_complete)

completed_keys = [
    "completed_sequence_rewarded",
    "completed_sequence_unrewarded",
    "completed_sequence_reward_timeout",
]
completed_dfs = [cls[k] for k in completed_keys if k in cls and isinstance(cls[k], pd.DataFrame)]
completed_df = pd.concat(completed_dfs, ignore_index=True) if completed_dfs else pd.DataFrame()

def _trial_id(row):
    for k in ["trial_id", "trial_index", "sequence_index", "Trial", "Sequence", "trial"]:
        if k in row and pd.notna(row[k]):
            return row[k]
    return row.name  # fallback

hits = []
for _, row in completed_df.iterrows():
    pos_pokes = row.get("position_poke_times")
    if not isinstance(pos_pokes, dict):
        continue
    tid = _trial_id(row)
    for pos, info in pos_pokes.items():
        if not isinstance(info, dict):
            continue
        ms = info.get("poke_time_ms")
        if ms is None:
            continue
        if ms < threshold_ms:
            odor = info.get("odor_name")
            hits.append((ms, tid, pos, odor))

# Print results sorted by poke time
if not hits:
    print(f"No completed-trial positions with poke_time_ms < {threshold_ms} ms found.")
else:
    hits.sort(key=lambda x: x[0])
    print(f"Trials with poke_time_ms < {threshold_ms} ms (n={len(hits)}):")
    for ms, tid, pos, odor in hits:
        print(f"- trial {tid}, position {pos}, {odor}: {ms:.1f} ms")

In [ ]:
# Can be used to test if Hidden Rule Odor Identities can be detected from the Schema file. 
session_settings, schema_settings = detect_settings.detect_settings(root)
print("HR index inferred:", schema_settings.get("hiddenRuleIndexInferred"))
print("HR odors inferred:", schema_settings.get("hiddenRuleOdorsInferred"))